<a href="https://colab.research.google.com/github/bhavesh-choudhary1/machine_learning_concept/blob/main/Sklearn_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

# Without using pipelines

In [ ]:
df = pd.read_csv('/content/train.csv')
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
253,254,0,3,"Lobb, Mr. William Arthur",male,30.0,1,0,A/5. 3336,16.1000,NaN,S
268,269,1,1,"Graham, Mrs. William Thompson (Edith Junkins)",female,58.0,0,1,PC 17582,153.4625,C125,S
242,243,0,2,"Coleridge, Mr. Reginald Charles",male,29.0,0,0,W./C. 14263,10.5000,NaN,S


In [ ]:
df.sample(4)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
678,0,3,female,43.0,1,6,46.9000,S
363,0,3,male,35.0,0,0,7.0500,S
723,0,2,male,50.0,0,0,13.0000,S
175,0,3,male,18.0,1,1,7.8542,S


In [ ]:
# train_test_split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                  df['Survived'],
                                                  test_size=0.2,
                                                  random_state=42)

In [ ]:
x_train.shape,x_test.shape

((712, 7), (179, 7))

In [ ]:
# Applying the imputer
age = SimpleImputer()
emb = SimpleImputer(strategy='most_frequent')

x_train_age = age.fit_transform(x_train[['Age']])
x_test_age = age.transform(x_test[['Age']])

x_train_emb = emb.fit_transform(x_train[['Embarked']])
x_test_emb = emb.transform(x_test[['Embarked']])

In [ ]:
# One Hot Encoding

ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_emb = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

x_train_sex = ohe.fit_transform(x_train[['Sex']])
x_test_sex = ohe.transform(x_test[['Sex']])

x_train_emb = ohe_emb.fit_transform(x_train_emb)
x_test_emb = ohe_emb.transform(x_test_emb)

In [ ]:
x_train_emb

array([[1., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1.],
       ...,
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1.]])

In [ ]:
x_train_rem = x_train.drop(columns=['Sex','Age','Embarked'])
x_test_rem = x_test.drop(columns=['Sex','Age','Embarked'])

In [ ]:
x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_emb),axis=1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_emb),axis=1)

In [ ]:
x_train_transformed.shape

(712, 13)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [ ]:
y_pred = clf.predict(x_test_transformed)

In [ ]:
acurracy = accuracy_score(y_test,y_pred)
acurracy*100

78.2122905027933

In [ ]:
import pickle

In [ ]:
import pickle

# Define the path to your Google Drive folder
save_path = '/content/drive/My Drive/models/'

# Save the models
pickle.dump(ohe, open(save_path + 'ohe.pkl', 'wb'))
pickle.dump(ohe_emb, open(save_path + 'ohe_emb.pkl', 'wb'))
pickle.dump(clf, open(save_path + 'clf.pkl', 'wb'))


# With Sklearn Pipelines

In [ ]:
df = pd.read_csv('/content/train.csv')
df.sample(3)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
158,159,0,3,"Smiljanic, Mr. Mile",male,NaN,0,0,315037,8.6625,NaN,S
90,91,0,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,NaN,S
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S


In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [ ]:
df.sample(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
187,1,1,male,45.0,0,0,26.5500,S
268,1,1,female,58.0,0,1,153.4625,S
691,1,3,female,4.0,0,1,13.4167,C


In [ ]:
# Train Test Split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                  df['Survived'],
                                                  test_size=0.2,
                                                  random_state=42)
x_train.shape,x_test.shape

((712, 7), (179, 7))

In [ ]:
# inputation transformer

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [ ]:
# One hot encoding

trf2 = ColumnTransformer([
    ('ohe_sex_emb',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6]),
    ],remainder='passthrough')

In [ ]:
# Scaling

trf3  = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [ ]:
# Feature Selection

# trf4 = SelectKBest(score_func=chi2,k=8)

In [ ]:
# Training the model

trf5 = DecisionTreeClassifier()

## Creating the pipeline

In [ ]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    #('trf4',trf4),
    ('trf5',trf5)
])

In [ ]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_emb',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf5', DecisionTreeClassifier())])

## Explore the Pipeline

In [ ]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [ ]:
# Predict

y_pred = pipe.predict(x_test)

In [ ]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [ ]:
acurracy = accuracy_score(y_test,y_pred)
acurracy*100

62.56983240223464

## Cross Validation using Pipeline

In [ ]:
# Cross validation using cross_val_score
from sklearn.model_selection import cross_val_score

cross_val_score(pipe ,x_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

## GridSearch using Pipeline

In [ ]:
# Grid Search cv

params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_emb',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [ ]:
grid.best_score_

0.6391214419383433

In [ ]:
grid.best_params_

{'trf5__max_depth': 2}

In [ ]:
# Export

import pickle

pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [ ]:
test_input2 = np.array([2,'male',31.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

test_input_pd = pd.DataFrame(test_input2,columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
)

In [ ]:
pipe.predict(test_input_pd)

array([0])